## Create an interactive report

* Choose a dataset! You can use any dataset for this task. Here is a list of some real life data:
  * https://data.boston.gov/organization 
  * https://catalog.data.gov/dataset/demographic-statistics-by-zip-code-acfc9
  * https://catalog.data.gov/dataset/crimes-2001-to-present-398a4
  * https://catalog.data.gov/dataset/lottery-mega-millions-winning-numbers-beginning-2002
  * https://catalog.data.gov/dataset/traffic-violations-56dda
  * or from any other source
* Create 5 different plots using **bokeh** or **plotly**

When you finished with the notebook, then convert it to html!

` jupyter-nbconvert --execute worksheet-interactive.ipynb
`

You can find further interactive tools on the *pyviz* site: 
* http://pyviz.org/
* http://holoviews.org

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, ColumnDataSource

from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool

from bokeh.models import FactorRange


import math

from bokeh.models import GraphRenderer, Oval, StaticLayoutProvider
from bokeh.palettes import Spectral8

import networkx as nx
from bokeh.models import Circle
from bokeh.models.graphs import from_networkx
from bokeh.transform import linear_cmap


In [2]:
df_demog = pd.read_csv('Demographic_Statistics_By_Zip_Code.csv')
df_lottery = pd.read_csv('Lottery_Mega_Millions_Winning_Numbers__Beginning_2002.csv')


df_corona = pd.read_csv('NxLoCeWZ.csv')
df_corona['date'] = pd.to_datetime(df_corona['dateRep'],dayfirst=True)
df_corona = df_corona.sort_values(by= ['date'])

In [3]:
df_corona.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continentExp,date
17617,31/12/2019,31,12,2019,0,0,Romania,RO,ROU,19473936.0,Europe,2019-12-31
13015,31/12/2019,31,12,2019,0,0,Luxembourg,LU,LUX,607728.0,Europe,2019-12-31
22217,31/12/2019,31,12,2019,0,0,United_States_of_America,US,USA,327167434.0,America,2019-12-31
20322,31/12/2019,31,12,2019,0,0,Sweden,SE,SWE,10183175.0,Europe,2019-12-31
8340,31/12/2019,31,12,2019,0,0,Germany,DE,DEU,82927922.0,Europe,2019-12-31


In [4]:
continent_names = list(df_corona.groupby('continentExp',as_index=False).sum().continentExp)
continent_deaths = list(df_corona.groupby('continentExp',as_index=False).sum().deaths)
continent_cases = list(df_corona.groupby('continentExp',as_index=False).sum().cases)


In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
data1 = {'Nr. of death': continent_deaths,
         'Nr. of cases': continent_cases,
         'label': continent_names}

source1 = ColumnDataSource(data=data1)
hover = HoverTool(tooltips=[('Continent', '@label')])
tools1 = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]


fig1 = figure(tools = tools1,
              title='Continents nr death & cases (2020. 06. 12.)',
              plot_height = 450, plot_width = 450,
              x_axis_label='Nr. of cases',
              y_axis_label='Nr. of deaths')
fig1.circle(x='Nr. of cases', y='Nr. of death', source=source1,size = 16, color='red')
show(fig1)

In [7]:
# df_corona[df_corona.continentExp == 'Europe'].groupby(by='date',as_index=False).sum()

In [8]:
df_corona_eu = df_corona[df_corona.continentExp == 'Europe'].groupby(by='date',as_index=False).sum()
df_corona_us = df_corona[df_corona.continentExp == 'America'].groupby(by='date',as_index=False).sum()

eu_time = list(df_corona_eu.date)
eu_cases = list(df_corona_eu.cases)
eu_deaths = list(df_corona_eu.deaths)

us_time = list(df_corona_us.date)
us_cases = list(df_corona_us.cases)
us_deaths = list(df_corona_us.deaths)

c_time = list(df_corona.date)
c_cases = list(df_corona.cases)
c_deaths = list(df_corona.deaths)

In [9]:
# data2 = {'Nr. of death': continent_deaths,
#          'Nr. of cases': continent_cases,
#          'label': continent_names}

# source2 = ColumnDataSource(data=data2)

# hover = HoverTool(tooltips=[('Continent', '@label')])
tools2 = [WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]


fig2 = figure(tools = tools2,
              title='Europe cases (2020. 06. 12.)',
              plot_height = 450, plot_width = 450,
              x_axis_label='Nr. of cases',
              y_axis_label='Nr. of deaths',
              x_axis_type='datetime',
              y_axis_type="log")



# fig2.circle(x='Nr. of cases', y='Nr. of death', source=source2,size = 16, color='red')
fig2.line(eu_time, eu_cases, legend="Cases", line_width=2)
fig2.line(eu_time, eu_deaths, legend="Deaths", line_width=2,color='red')

# fig2.line(eu_time, eu_cases, legend_label="Cases", line_width=2)
# fig2.line(eu_time, eu_deaths, legend_label="Deaths", line_width=2,color='red')

show(fig2)

In [10]:
data3 = {'names' : continent_names,
        'Cases'   : continent_cases}

source3 = ColumnDataSource(data3)
fig3 = figure(tools = tools2,
              x_range=continent_names, plot_height=250,
              title="Continent's nr of  cases",
              x_axis_label = 'Continents',
              y_axis_label = 'Nr. of cases')
fig3.vbar(x='names', top='Cases', width=0.9,source=source3)
fig3.add_tools(HoverTool(tooltips=[("Cases", "@Cases")]))

fig3.xgrid.grid_line_color = None
fig3.y_range.start = 0

show(fig3)

In [11]:
case_type = ['cases', 'deaths']

data = {'Continents' : continent_names,
        'cases'   : continent_cases,
        'deaths'   : continent_deaths}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in continent_names for year in case_type ]
counts = sum(zip(data['cases'], data['deaths']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

hover = HoverTool(tooltips=[('Cases', '@Continents')])
tools4 = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]


p = figure(x_range=FactorRange(*x), plot_height=250, title="Data by continents",
           toolbar_location='below',tools=tools2,
           y_axis_label = 'Counter')

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white")

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [12]:

G = nx.les_miserables_graph()

fig5 = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools=tools2)

graph = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))
graph.node_renderer.data_source.data['name'] = list(G.nodes())
graph.node_renderer.glyph = Circle(size=10,
                                   fill_color=linear_cmap('name', 'Spectral8', 1, 200))
fig5.renderers.append(graph)

show(fig5)